In [21]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from fiona.crs import from_epsg

In [22]:
gdf_buildings = gpd.read_file('gangnamGu-building-residential.geojson')

In [23]:
gdf_buildings = gdf_buildings.loc[~gdf_buildings['UFID'].duplicated()]

In [24]:
gdf_buildings_buffer = gdf_buildings.copy() 

In [25]:
gdf_buildings_buffer.crs = from_epsg(4326)
gdf_buildings_buffer = gdf_buildings_buffer.to_crs(epsg=5174)

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [26]:
# make a buffer zone. radius = 900ft
gdf_buildings_buffer['buffer_geometry_400m'] = gdf_buildings_buffer['geometry'].buffer(400)

In [27]:
gdf_bus = gpd.read_file('seoulBusStop.geojson')

In [28]:
gdf_bus.crs = from_epsg(4326)
gdf_bus = gdf_bus.to_crs(epsg=5174)

C:\Users\hsg82\AppData\Local\Continuum\anaconda3\envs\data_basic\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [29]:
gdf_bus.head()

,표준ID,ARS-ID,정류장명,X좌표,Y좌표,geometry
0,100000001,1001,종로2가사거리,126.987750,37.569765,POINT (198848.178 451942.074)
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,POINT (199627.083 452987.282)
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,POINT (199783.807 453374.424)
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,POINT (198836.075 451810.459)
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,POINT (200084.468 453770.855)


In [30]:
def count_busStop(buffer_zone):
    '''
    This function is for making a list of bus stops that is in a buffer zone.
    -----
    input:
    buffer_zone (Shapely.geometry.Polygon): a polygon of a buffer zone 
    -----
    output:
    precise_matches_list (list): a list of the bus stops
    
    '''
    try:
        # filter 511 events based on buffer zone boundary of a 511 events
        stops_sindex = gdf_bus.sindex
        possible_matches_index = list(stops_sindex.intersection(buffer_zone.bounds))
        possible_matches = gdf_bus.iloc[possible_matches_index]
        # extract the list of crash that matched with a 511 event
        precise_matches_list = possible_matches[possible_matches.intersects(buffer_zone)]['표준ID'].tolist()
    except:
        # if there is no crashes that matched with temporal range of a 511 event, return empty list
        precise_matches_list = []
    return precise_matches_list

In [31]:
gdf_buildings_buffer['bus_list_400m']= gdf_buildings_buffer['buffer_geometry_400m'].apply(lambda x:count_busStop(x))

In [32]:
gdf_buildings_buffer['count_bus_stop'] = gdf_buildings_buffer['bus_list_400m'].apply(lambda x:len(x))

In [37]:
gdf_buildings = gdf_buildings.merge(gdf_buildings_buffer[['UFID','count_bus_stop']], on='UFID')

In [38]:
gdf_buildings.head()

,USABILITY,HEIGHT,PNU,GRND_FLR,UGRND_FLR,DONG_NM,UFID,BC_RAT,VC_RAT,COL_ADM_SECT_CD,BLDG_PNU,OBJECTID,BD_MGT_SN,geometry,count_bus_stop
0,01000,7.90,1168011400106410010,2,1,None,1987207556994435369900000000,49.21,None,None,None,None,1168011400106410010001998,"MULTIPOLYGON (((127.08634 37.49399, 127.08628 ...",9
1,02000,13.16,1168010800101570024,4,1,None,2003202386534451790800000000,54.62,None,None,None,None,1168010800101570024008663,"MULTIPOLYGON (((127.02788 37.50883, 127.02781 ...",2
2,02000,0.00,1168010500100790000,12,1,6,1981205226284463626200000000,0.00,None,None,None,None,1168010500100790000014655,"MULTIPOLYGON (((127.06025 37.51966, 127.06031 ...",5
3,01000,9.80,1168010400100110005,3,0,´Ù°¡±¸¿ë´Üµ¶ÁÖÅÃ(9°¡±¸),1995204058644466407200000000,59.76,None,None,None,None,1168010400100110005018473,"MULTIPOLYGON (((127.04673 37.52192, 127.04665 ...",10
4,01000,0.00,1168010500100700004,2,1,None,1978204624064458510800000000,0.00,None,None,None,None,1168010500100700004015465,"MULTIPOLYGON (((127.05317 37.51489, 127.05315 ...",6


In [39]:
gdf_buildings.to_file('gangnamGu-building-residential.geojson', encoding='utf8', driver='GeoJSON')

In [40]:
gdf_buildings['count_bus_stop'].max()

32